# Machine learning case study: rain in Australia dataset
### Anna Przybyłowska, Gurbet Gungoren, Wojciech Tomczak, Witold Taisner

## 1. Used libraries

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA

############ MODELS ############################

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

#################################################

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from collections import Counter

## 2. Importing preprocessed data

We decided to use one-hot encoding, as it performed slightly better than label-encoding

In [2]:
df = pd.read_csv("data/rain_outliers_removed.csv")

# encoding RainTomorrow and RainToday as binary values
df.RainToday.replace(("Yes", "No"), (1,0), inplace = True)
df.RainTomorrow.replace(("Yes", "No"), (1,0), inplace = True)

#################### ONE-HOT ENCODING #########################################

# columns to be changed to one-hot encoding
categorical_columns = ["Season", "WindGustDir", "WindDir9am", "WindDir3pm"]

# creating one-hot encoding
df = pd.get_dummies(df, columns = categorical_columns)

#################### LABEL ENCODER ############################################

# le = LabelEncoder()

# df["Season"] = le.fit_transform(df["Season"])
# df["WindDir9am"]= le.fit_transform(df["WindDir9am"])
# df["WindDir3pm"]= le.fit_transform(df["WindDir3pm"])
# df["WindGustDir"] = le.fit_transform(df["WindGustDir"])

###############################################################################

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,13.4,22.9,0.6,7.0,10.5,44,20,24,71,22,...,0,0,0,0,0,0,0,0,1,0
1,7.4,25.1,0.0,7.6,13.3,44,4,22,44,25,...,0,0,0,0,0,0,0,0,0,1
2,12.9,25.7,0.0,11.4,10.0,46,19,26,38,30,...,0,0,0,0,0,0,0,0,0,1
3,9.2,28.0,0.0,6.8,12.2,24,11,9,45,16,...,0,0,0,0,0,0,0,0,0,0
4,17.5,32.3,1.0,8.0,5.0,41,7,20,82,33,...,0,1,0,0,0,0,0,0,0,0


In [3]:
df.shape

(123710, 70)

In [4]:
y = df.RainTomorrow.to_numpy()
X = df.drop(columns=['RainTomorrow']).to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## 3. Used metrics:
Apart from standard accuracy, we decided to also evaluate our models based on [balanced accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html), which is better suited for inbalanced data, as well as F1, Precision and Recall.

## 4. Tested approaches:
We decided to test 4 approaches, as our data is quite imbalanced.
***
### 4.1 Training on preprocessed only dataset
We divided our data into two parts, training set consisting of 80% and testing set consisting of 20% of original data.

- SVC:
    - StandardScaler: accuracy: 85%, balanced accuracy: 72%;
- KNN:
    - StandardScaler: accuracy: 80%, balanced accuracy: 64%;
- MLP Classifier:
    - StandardScaler: accuracy: 83%, balanced accuracy: 72%;
- Decision Tree Classifier:
    - StandardScaler: accuracy: 78%, balanced accuracy: 69%;
- **Random Forest Classifier**:
    - StandardScaler: accuracy: 85%, balanced accuracy: 72%; // 100 estimators
    - StandardScaler: accuracy: 85%, balanced accuracy: 72%; // 300 estimators
- **AdaBoost Classifier**:
    - StandardScaler: accuracy: 84%, balanced accuracy: 71%; // 100 estimators
    - StandardScaler: accuracy: 84%, balanced accuracy: 72%; // 300 estimators
- **XGBoost Classifier**:
    - StandardScaler: accuracy: 85%, balanced accuracy: 74%;
- Logistic Regression:
    - StandardScaler: accuracy: 85%, balanced accuracy: 72.7%;
- **LGBMClassifier**:
    - StandardScaler: accuracy: 85.3%, balanced accuracy: 73.5%;
    
We managed to determine four classifiers, written in bold case, which manged to get the best results on this data.
***
### 4.2 Oversampling

Second of tested approaches focused only on previously found classifiers: *XGBoost*, *Random Forest*, *LGBM* and some of the more promising: *AdaBoost*. 
Oversampling creates copies of minority class, so that there is even number of each class instance.

- **Random Forest Classifier**:
    - StandardScaler: accuracy: 85.2%, balanced accuracy: 74.3%; 
- LGBMClassifier:
    - StandardScaler: accuracy: 85.2%, balanced accuracy: 73.5%;
- XGBoost:
    - StandardScaler: accuracy: 85.2%, balanced accuracy: 73.8%;
- AdaBoost:
    - StandardScaler: accuracy: 81.4%, balanced accuracy: 75%;
    
Easy to notice, oversampling did not improve our results in a significant way.

***
### 4.3 Undersampling
Similarly to oversampling, we focused on *XGBoost*, *Random Forest*, *LGBM* and *AdaBoost*. Undersampling is a method of removing similar instances of majority class, so that its cardinality is the same as minority class.

- **Random Forest Classifier**:
    - StandardScaler: accuracy: 79%, balanced accuracy 79%;
- LGBMClassifier:
    - StandardScaler: accuracy: 79.1%, balanced accuracy: 78.9%;
- **XGBoost**:
    - StandardScaler: accuracy: 79%, balanced accuracy: 79%;
- AdaBoost:
    - StandardScaler: accuracy: 78.2%, balanced accuracy: 77.4%;
    
Undersampling decreases overall accuracy, but at the same time increases balanced accuracy (better prediction of minority class)

***
### 4.4 Feature selection, grid search, class weights
In addition we tested some other approaches:
        
#### 4.4.1 Grid search 
##### (only for Random Forest and XGBoost)
We tried to determine the best parameters for Random Forest and XGBoost with GridSerchCV method connected with K-best feature selection. It managed to determine best parameters and correspodning results:

- RandomForest: 
    - 'criterion': 'entropy',
    - 'max_depth': None,
    - 'min_samples_leaf': 4,
    - 'n_estimators': 100,
    - 'feature_selection k': 20
    - StandardScaler: **accuracy: 85%, balanced accuracy: 72%**;
- XGBoost Classifier:
    - 'colsample_bytree': 0.6,
    - 'gamma': 0,
    - 'max_depth': 8,
    - 'min_child_weight': 2,
    - 'subsample': 1.0,
    - 'feature_selection k': 40}
    - StandardScaler: **accuracy: 85.2%, balanced accuracy: 73.6%**;
    
Grid searches managed to get similar results as training on dataset only.
        
#### 4.4.2 Class weights
In this approach we assigned weights to each class, so that the model would maximize its objective function with minority class having bigger weight.
- **XGBoost Classifier**:
    - StandardScaler: accuracy: 81%, balanced accuracy 79.4%;
- Random Forest Classifier:
    - StandardScaler: accuracy: 85.1%, balanced accuracy 71.5%;
- LGBMClassifier:
    - StandardScaler: accuracy: 80.1%, balanced accuracy: 79.6%;
    
Assigning weights to classes seems to produce the best trade-off between overall accuracy and balanced accuracy.

## 5. Implementation

In [5]:
pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
#         ('feature_selection', SelectKBest(f_classif, k = 20)) ,
#         ('pca', PCA(0.95)),
        ('classifier', XGBClassifier(use_label_encoder=False, scale_pos_weight = sum(y_train == 0)/sum(y_train == 1)))

    ], 
    verbose=True
    ) 
    

Parameters to be checked by GridSearchCV

Training the model

In [6]:
%%time
pipe.fit(X_train, y_train)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.2s
[13:30:14] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   7.9s
CPU times: user 1min 30s, sys: 157 ms, total: 1min 30s
Wall time: 8.11 s


Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=12, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=3.5120816996443875, subsample=1,
                               tree_method='exact', use_label_encoder=False,
                               validate_parameters=1, verbosity=None))],
    

Prediction:

In [7]:
y_predicted = pipe.predict(X_test)

Report:

In [8]:
report = metrics.classification_report(y_test, y_predicted)
print(report)
print("Accuracy of the model is:",metrics.accuracy_score(y_test,y_predicted)*100,"%")
print("Balanced accuracy of the model is:",metrics.balanced_accuracy_score(y_test, y_predicted)*100,"%")
cm = metrics.confusion_matrix(y_test, y_predicted)
cm

              precision    recall  f1-score   support

           0       0.93      0.82      0.87     19284
           1       0.55      0.77      0.64      5458

    accuracy                           0.81     24742
   macro avg       0.74      0.79      0.75     24742
weighted avg       0.84      0.81      0.82     24742

Accuracy of the model is: 80.9514186403686 %
Balanced accuracy of the model is: 79.359795406213 %


array([[15853,  3431],
       [ 1282,  4176]])